<h1 align='center'>Welcome to my Notebook!<h1>

Previously, I have described and implemented batch and minibatch gradient descent, with their respenctive pros and cons.

In this kernel, I would like to discuss another powerful gradient descent variation: Stochastic gradient descent.

I hope you enjoy and make sure to upvote if you did!

In [ ]:
import numpy as np
import pandas as pd
from sklearn.linear_model import SGDRegressor
from sklearn.datasets import load_boston
from sklearn.metrics import mean_squared_error

As in the previous dataset, I will be using the boston housing data that is prebuilt into `sklearn` 

In [ ]:
# Load and convert data to DataFrame
data = load_boston()

df = pd.DataFrame(data['data'],columns=data['feature_names'])
df.insert(13,'target',data['target'])
df.head(5)

<h1 align='center'> Gradient Descent Explained </h1>

Before we dive in to Stochastic Gradient Descent, You need to understand what gradient descent itself is. 

Essentially, gradient descent is an iterative optimization algorithm that aims to get the minimum of a function.

In other words, if you had a cost function(MSE,MAE,RMSE) and it is quite high, then the objective of gradient descent is to either reduce it greatly to a small value(local minimum), or reduce in completely(global minimum). 

It works by iteratively looping through the dataset and substracting the partial derivative from the current coefficent value times the alpha(learning rate). The formula is shown below:


![](https://i.stack.imgur.com/3MhPr.png)

You can think of gradient descent like this: if I was on top of a mountain, what is the best direction to take a step in? 


<h2>Learning Rate</h2>

The learning rate controls the size of each 'step' taken by gradient descent.

If the learning rate is too large, the gradient descent might miss the global minimum. If the learning rate is too small, then gradient descent wil take to long to converge(reach a point where it is no longer descreasing). 

![](https://cdn-images-1.medium.com/max/1000/1*An4tZEyQAYgPAZl396JzWg.png)

<h2>Types of Gradient Descent</h2>


There are three main types of Gradient Descent:

1. Batch Gradient Descent
2. Stochastic Gradient Descent
3. Mini-batch gradient Descent

**Batch Gradient Descent**

Here, the **whole** training set(batch) is taken into consideration when caclulating the derivatives. While this may have the smoothest path, it can become extremely slow and computationally expensive to do so. Therefore, it is not ideal for large datasets.

**Stochastic Gradient Descent**

Here, instead of calculating the partial derivative for the whole training set, the calcuation is only done on one **random** sample(stochastic meaning random). This is great because the calcuations are only needed to be done on one training example instead of the whole training set, making it much faster and ideal for large datasets.

However,due to its inherent randomness, stochastic gradient descent does not have a smooth descend as in batch gradient descent. This means that it will bounce around, and while it may produce good parameters,they will rarely be optimal.

**Mini-batch Gradient Descent**

Instead of computing the gradients based on the whole training set,or just one training set, it computes it on small random subsets of the training set called **mini batches**.

It tends to have a much smoother descent than stochastic gradient descent, and is again computationally fast. However, like stochastic gradient descent, it may be stuck on local minima, meaning that the parameters will be good, but not optimal

![](https://i.stack.imgur.com/lU3sx.png)


As suggested by the title, we will be implementing stochastic gradient descent, with a twist!

An important thing to note is that the initial parameters are randomly initialized. This is known as (you guessed it), Random Initialization!

In [ ]:
X,y = df.drop('target',axis=1),df['target']

thetas = np.zeros(X.shape[1])

<h1 align='center'>Cost Functions</h1>

a cost function,or a loss function, is simply a function that calculates the loss of a hypothesis. These are usually names Evaluation Metrics on Kaggle. Commons ones include:

MSE(Mean Squared Error)
RMSE(Root Mean Squared Error)
MAE(Mean Absolute Error)

Different cost functions have different pros and cons, but here I will be implementing Mean Squared Error.

![](https://miro.medium.com/max/808/1*-e1QGatrODWpJkEwqP4Jyg.png)

In [ ]:
def cost_function(X,Y,B):
    predictions = np.dot(X,B.T)
    
    cost = (1/len(Y)) * np.sum((predictions - Y) ** 2)
    return cost

Ok, so let's test the cost function by using `sklearn's` mean_sqaured_error function to see if we implemented it correctly!

In [ ]:
cost_function(X,y,thetas)

In [ ]:
mean_squared_error(np.dot(X,thetas.T),y)

Looks Good!

<h1 align='center'>Stochastic Gradient Descent Implementation</h1>

Ok, now let's get into the fun stuff ;)

<h1 align='center'>Scaling</h1>

In order for gradient descent to operate properly, we need to scale our features to that they are on a similar scale. This ensures that all the features are on a similiar scale, and helps gradient descent converge quicker and also reduces computational time. 

I will be using Min-Max Scaling to scale my features.

![](https://media.geeksforgeeks.org/wp-content/uploads/min-max-normalisation.jpg)

In [ ]:
X_norm = (X - X.min()) / (X.max() - X.min())
X = X_norm

<h1 align='center'>Learning Schedule</h1>

Because of the inherent randomness of Stochastic Gradient Descent, setting the learning rate can prove to be quite a difficult task as the algorithm can never settle at a minimum.

One solution is to set the learning rate to be initially large(so it can bypass local optima) and then gradually shrink, giving the algorithm a better chance at reaching global minimum.

This process is known as a *learning schedule*. We will code a simple learning schedule for our algorithm:

In [ ]:
t0,t1 = 5,50 # learning schedule hyperparams
def learning_schedule(t):
    return t0/(t+t1)

Now the moment you have been waiting for.. Stochastic Gradient Descent from Scratch!

In [ ]:
def stochastic_gradient_descent(X,y,theta,n_epochs=50):
    c_hist = [0] * n_epochs
    for epoch in range(n_epochs):
        for i in range(len(y)):
            rand_index = np.random.randint(len(y))
            ind_x = X[rand_index:rand_index+1]
            ind_y = y[rand_index:rand_index+1]

            gradients = 2 * ind_x.T.dot(ind_x.dot(theta) - ind_y)
            eta = learning_schedule(epoch * len(y) + i)
            theta = theta - eta * gradients
            c_hist[epoch] = cost_function(ind_x,ind_y,theta)
    return theta,c_hist

In [ ]:
th_n,cost_history = stochastic_gradient_descent(X,y,thetas)

Ok, now let's evaluate to see how well the algorithm performed

In [ ]:
mean_squared_error(np.dot(X,th_n.T),y)

Wow! From 592 to 29, definitely an improvement! Let's see a line plot of the cost function by the number of epochs.

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(range(50),cost_history)
plt.show()

Thanks for reading, I hope you enjoyed it and learned something new. Make sure to upvote!